### 주의
- 함수 현재 index 500까지 설정되어있습니다. (테스트) 
- `get_station_across` 3번 주석 해제해주세요 (전체)

In [1]:
# 추가되는 패키지는 여기에서 import 해주세요
import warnings
warnings.filterwarnings('ignore')

import missingno
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import geopy.distance
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)

import numpy as np
import datetime
import statsmodels.api as sm

In [2]:
# 데이터 load
train = pd.read_csv('train.csv', date_parser='date')
# 분석을 위해 컬럼명 수정
train.rename(columns={'6~7_ride': 'ride6', '7~8_ride': 'ride7', '8~9_ride': 'ride8',\
                      '9~10_ride': 'ride9', '10~11_ride': 'ride10', '11~12_ride': 'ride11', 
                      '6~7_takeoff': 'off6', '7~8_takeoff': 'off7', '8~9_takeoff': 'off8', \
                      '9~10_takeoff': 'off9', '10~11_takeoff': 'off10', '11~12_takeoff': 'off11',\
                      '18~20_ride': 'ride18'},inplace=True)


# 시작 날짜와, 끝 날짜 확인
print(train['date'].min())
print(train['date'].max())
# date 타입 변경
train['date'] = train['date'].astype('datetime64')
# 9월 1일 ~ 9월 30일 데이터이니까, day만 따로 가져옴
train['date_day'] = train['date'].dt.day
# 월(0) ~ 일(6) 데이터로 변환
train['date_dayofweek'] = train['date'].dt.dayofweek
train['date_dayofname']= train['date'].dt.day_name()
train[['date','date_dayofweek','date_day','date_dayofname']]

#평일과 휴일을 나누기, 금토일 =1, 나머지는 0
train.loc[train['date_dayofweek'] == 0, 'holiday' ] = 0
train.loc[train['date_dayofweek'] == 1, 'holiday' ] = 0
train.loc[train['date_dayofweek'] == 2, 'holiday' ] = 0
train.loc[train['date_dayofweek'] == 3, 'holiday' ] = 0
train.loc[train['date_dayofweek'] == 4, 'holiday' ] = 1
train.loc[train['date_dayofweek'] == 5, 'holiday' ] = 1
train.loc[train['date_dayofweek'] == 6, 'holiday' ] = 1
# 추석 추가도 공휴일로 처리
train.loc[train['date'] == '2019-09-14', 'holiday'] = 1
train.loc[train['date'] == '2019-09-13', 'holiday'] = 1
train.loc[train['date'] == '2019-09-12', 'holiday'] = 1


# 시내와 시외를 int형으로 원핫인코딩
train.loc[train['in_out'] == '시외','in_out'] = 1
train.loc[train['in_out'] == '시내','in_out'] = 0

train.tail()

2019-09-01
2019-09-30


,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,ride6,ride7,ride8,ride9,ride10,ride11,off6,off7,off8,off9,off10,off11,ride18,date_day,date_dayofweek,date_dayofname,holiday
415418,415418,2019-09-30,32820000,0,1129,한림환승정류장(한림리),33.41437,126.26336,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0
415419,415419,2019-09-30,32820000,0,1564,제주시외버스터미널,33.49946,126.51479,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0
415420,415420,2019-09-30,32820000,0,2322,해병부대,33.23100,126.26273,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0
415421,415421,2019-09-30,32820000,0,3291,애월환승정류장(애월리),33.46483,126.31870,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0
415422,415422,2019-09-30,32820000,0,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0


In [3]:
total = train.groupby(['station_name', 'station_code']).count()
total

id  date  bus_route_id  in_out  latitude  \
station_name station_code                                              
(구)구판장       4211          33   33    33            33      33         
             4212          10   10    10            10      10         
(구)중문동주민센터   2058          263  263   263           263     263        
             2059          299  299   299           299     299        
(구)중앙파출소     1952          280  280   280           280     280        
             2080          177  177   177           177     177        
             2696          76   76    76            76      76         
(구)화산초등학교    2146          6    6     6             6       6          
             4210          18   18    18            18      18         
1100고지휴게소    1245          18   18    18            18      18         
             2638          25   25    25            25      25         
1100도로입구     1653          74   74    74            74      74         
4.3평화공원      1526          44   44    44            44      44         
             1527          10   10    10            10      10         
4.3평화공원입구    1529          14   14    14            14      14         
6호광장         694           162  162   162           162     162        
             695           66   66    66            66      66         
911의원        1243          46   46    46            46      46         
             1244          84   84    84            84      84         
GM빌라         538           61   61    61            61      61         
             539           68   68    68            68      68         
KIS빌리지       4086          5    5     5             5       5          
             4087          18   18    18            18      18         
LH12단지       2767          89   89    89            89      89         
             2768          124  124   124           124     124        
LH아파트        2083          464  464   464           464     464        
L마트          1937          413  413   413           413     413        
NHF삼화1단지     3510          379  379   379           379     379        
NLCS제주       2741          74   74    74            74      74         
             2742          72   72    72            72      72         
...                        ..   ..    ..            ..      ..         
회수마을회관       3073          137  137   137           137     137        
회수사거리        3590          22   22    22            22      22         
             3591          16   16    16            16      16         
회수삼거리        1649          69   69    69            69      69         
             1650          39   39    39            39      39         
회천입구         263           151  151   151           151     151        
효돈농협하나로마트    3389          93   93    93            93      93         
             3390          35   35    35            35      35         
효돈중학교        1817          248  248   248           248     248        
             1818          317  317   317           317     317        
효돈초등학교       1819          357  357   357           357     357        
             2072          259  259   259           259     259        
효돈축구공원       3387          22   22    22            22      22         
             3388          4    4     4             4       4          
효례교          2075          85   85    85            85      85         
효성빌라         1799          61   61    61            61      61         
             1800          53   53    53            53      53         
휴애리자연생활공원    3614          11   11    11            11      11         
             3615          20   20    20            20      20         
흙담솔사가        3478          23   23    23            23      23         
             3479          39   39    39            39      39         
흙통           1922          34   34    34            34      34         
             1923          46   46    46            46      46      

In [4]:
station = {}
for idx in total.index:
    if idx[0] not in station.keys():
        station[idx[0]] = 1
    else:
        station[idx[0]] += 1

df = pd.DataFrame()
df['station_name'] = station.keys()
df['counts'] = station.values()
df = df.sort_values(['counts','station_name'], ascending = [False, True])
df.head()

,station_name,counts
1226,신안동,8
781,금악리,7
559,비석거리,6
1667,성읍1리,6
1275,산방산,5


In [5]:
total2 = train.groupby(['station_name', 'station_code','bus_route_id']).count()
total2.sort_values('id',ascending = False, inplace = True)
total2

id  date  in_out  latitude  \
station_name   station_code bus_route_id                               
삼화지구           2894         23250000      30  30    30      30         
정존마을           431          23620000      30  30    30      30         
탐라장애인종합복지관     166          24310000      30  30    30      30         
정존마을           431          21480000      30  30    30      30         
탐라장애인종합복지관     166          23570000      30  30    30      30         
정존마을           431          22550000      30  30    30      30         
탐라장애인종합복지관     166          23490000      30  30    30      30         
                            23240000      30  30    30      30         
                            23220000      30  30    30      30         
                            23180000      30  30    30      30         
정존마을           431          22570000      30  30    30      30         
탐라장애인종합복지관     166          22520000      30  30    30      30         
                            28500000      30  30    30      30         
정존마을           431          29220000      30  30    30      30         
도련1동           375          29770000      30  30    30      30         
탐라장애인종합복지관     166          22240000      30  30    30      30         
                            22220000      30  30    30      30         
정존마을           431          31590000      30  30    30      30         
도련1동           375          29510000      30  30    30      30         
                            24010000      30  30    30      30         
                            23460000      30  30    30      30         
                            23250000      30  30    30      30         
정존마을           431          21420000      30  30    30      30         
정실마을           518          31480000      30  30    30      30         
정존마을           432          4270000       30  30    30      30         
정부제주지방합동청사     141          23510000      30  30    30      30         
도리초등학교         492          24220000      30  30    30      30         
               491          24240000      30  30    30      30         
                            24230000      30  30    30      30         
탑동입구           115          24020000      30  30    30      30         
...                                       ..  ..    ..      ..         
주거물            1732         30490000      1   1     1       1          
봉성하나로마트        1293         28120000      1   1     1       1          
시티주유소          446          29440000      1   1     1       1          
종합어시장          112          30240000      1   1     1       1          
남군농협           2063         25090000      1   1     1       1          
종합어시장          112          24030000      1   1     1       1          
남군농협           2063         26140000      1   1     1       1          
주공34단지         1802         28600000      1   1     1       1          
시외버스출발지(가상정류소) 1565         31330000      1   1     1       1          
남국원(아라방면)      3280         31320000      1   1     1       1          
부록마을           298          29660000      1   1     1       1          
남국원(아라방면)      3280         22490000      1   1     1       1          
부면동            814          22580000      1   1     1       1          
남국원(아라방면)      3280         22530000      1   1     1       1          
                            23530000      1   1     1       1          
                            23610000      1   1     1       1          
시외버스출발지(가상정류소) 1565         21300000      1   1     1       1          
부록마을           298          31830000      1   1     1       1          
남국원(아라방면)      3280         24080000      1   1     1       1          
주공6단지          2836         26060000      1   1     1       1          
부록마을           297          31830000      1   1     1       1          
               298          28940000      1   1     1       1          
주공34단지         2765         32770000      1   1     1       1          
시외버스출발지(가상

In [6]:
len(train.station_code.unique())

3563

In [7]:
dd = total2.query('id == 1')
dd.index.levels[0]
dd.query('station_name == "효성빌라"')


id  date  in_out  latitude  longitude  \
station_name station_code bus_route_id                                          
효성빌라         1799         30400000      1   1     1       1         1           
             1800         26170000      1   1     1       1         1           

                                        ride6  ride7  ride8  ride9  ride10  \
station_name station_code bus_route_id                                       
효성빌라         1799         30400000      1      1      1      1      1        
             1800         26170000      1      1      1      1      1        

                                        ride11  off6  off7  off8  off9  off10  \
station_name station_code bus_route_id                                          
효성빌라         1799         30400000      1       1     1     1     1     1       
             1800         26170000      1       1     1     1     1     1       

                                        off11  ride18  date_day  \
station_name station_code bus_route_id                            
효성빌라         1799         30400000      1      1       1          
             1800         26170000      1      1       1          

                                        date_dayofweek  date_dayofname  \
station_name station_code bus_route_id                                   
효성빌라         1799         30400000      1               1                
             1800         26170000      1               1                

                                        holiday  
station_name station_code bus_route_id           
효성빌라         1799         30400000      1        
             1800         26170000      1

In [8]:
total2.query('station_name == "효성빌라"')

id  date  in_out  latitude  longitude  \
station_name station_code bus_route_id                                          
효성빌라         1800         25050000      26  26    26      26        26          
             1799         25040000      23  23    23      23        23          
                          26150000      19  19    19      19        19          
             1800         26140000      19  19    19      19        19          
             1799         26170000      18  18    18      18        18          
             1800         26160000      7   7     7       7         7           
             1799         30400000      1   1     1       1         1           
             1800         26170000      1   1     1       1         1           

                                        ride6  ride7  ride8  ride9  ride10  \
station_name station_code bus_route_id                                       
효성빌라         1800         25050000      26     26     26     26     26       
             1799         25040000      23     23     23     23     23       
                          26150000      19     19     19     19     19       
             1800         26140000      19     19     19     19     19       
             1799         26170000      18     18     18     18     18       
             1800         26160000      7      7      7      7      7        
             1799         30400000      1      1      1      1      1        
             1800         26170000      1      1      1      1      1        

                                        ride11  off6  off7  off8  off9  off10  \
station_name station_code bus_route_id                                          
효성빌라         1800         25050000      26      26    26    26    26    26      
             1799         25040000      23      23    23    23    23    23      
                          26150000      19      19    19    19    19    19      
             1800         26140000      19      19    19    19    19    19      
             1799         26170000      18      18    18    18    18    18      
             1800         26160000      7       7     7     7     7     7       
             1799         30400000      1       1     1     1     1     1       
             1800         26170000      1       1     1     1     1     1       

                                        off11  ride18  date_day  \
station_name station_code bus_route_id                            
효성빌라         1800         25050000      26     26      26         
             1799         25040000      23     23      23         
                          26150000      19     19      19         
             1800         26140000      19     19      19         
             1799         26170000      18     18      18         
             1800         26160000      7      7       7          
             1799         30400000      1      1       1          
             1800         26170000      1      1       1          

                                        date_dayofweek  date_dayofname  \
station_name station_code bus_route_id                                   
효성빌라         1800         25050000      26              26               
             1799         25040000      23              23               
                          26150000      19              19               
             1800         26140000      19              19               
             1799         26170000      18              18               
             1800         26160000      7               7                
             1799         30400000      1               1                
             1800         26170000      1               1                

                                        holiday  
station_name station_code bus_route_id           
효성빌라         1800         25050000      26       
             1799         25040000      23       
                         

In [169]:
dd

,,,id,date,in_out,latitude,longitude,ride6,ride7,ride8,ride9,ride10,ride11,off6,off7,off8,off9,off10,off11,ride18,date_day,date_dayofweek,date_dayofname,holiday,dis_jejusi,dis_seoquipo
station_name,station_code,bus_route_id,,,,,,,,,,,,,,,,,,,,,,,,
삼양초등학교,221,23460000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
화북초등학교,425,29630000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
헬스케어타운,4173,26110000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
서귀포시구시외버스터미널,6103005,32090000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
삼무공원,342,23140000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
한림오일시장,966,27980000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
(구)중문동주민센터,2058,22550000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
고향모루,1947,30490000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
화북초등학교,425,29810000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [226]:
train.query('bus_route_id == 29630000')

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,ride6,ride7,ride8,ride9,ride10,ride11,off6,off7,off8,off9,off10,off11,ride18,date_day,date_dayofweek,date_dayofname,holiday,dis_jejusi,dis_seoquipo
8529,8529,2019-09-01,29630000,0,121,제주대학교병원,33.46909,126.54739,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,4.193535,23.275524
8530,8530,2019-09-01,29630000,0,123,인다마을,33.47145,126.54540,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,3.873265,23.545690
8531,8531,2019-09-01,29630000,0,125,아라주공아파트,33.47366,126.54542,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,3.670933,23.790402
8532,8532,2019-09-01,29630000,0,223,삼양2동,33.52128,126.58236,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,5.986448,29.129005
8533,8533,2019-09-01,29630000,0,268,화북남문,33.51890,126.56512,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,4.419615,28.785459
8534,8534,2019-09-01,29630000,0,274,동광양,33.50220,126.53222,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,0.893135,27.034478
8535,8535,2019-09-01,29630000,0,276,문예회관,33.50463,126.53691,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,1.382672,27.267726
8536,8536,2019-09-01,29630000,0,278,인화초등학교,33.50668,126.53937,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,1.676724,27.478943
8537,8537,2019-09-01,29630000,0,292,천수동,33.50934,126.54257,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,2.071545,27.756167
8538,8538,2019-09-01,29630000,0,468,제주동중학교,33.51958,126.57078,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,4.925220,28.877380


In [227]:
train.query('station_name == "화북초등학교" & bus_route_id == 29630000')

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,ride6,ride7,ride8,ride9,ride10,ride11,off6,off7,off8,off9,off10,off11,ride18,date_day,date_dayofweek,date_dayofname,holiday,dis_jejusi,dis_seoquipo
184294,184294,2019-09-14,29630000,0,425,화북초등학교,33.51762,126.56172,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14,5,Saturday,1.0,4.073772,28.638178


In [224]:
s = train.query('date == "2019-09-14" & bus_route_id == 29630000')
map_osm = folium.Map(location=[33.48944, 126.48508], zoom_start= 13)
for i in s.index:
    folium.Circle([s.loc[i,'latitude'], s.loc[i,'longitude']], radius = 1,color='red', popup= s.loc[i,'bus_route_id']).add_to(map_osm)
map_osm

In [225]:
s = train.query('date == "2019-09-15" & bus_route_id == 29630000')
map_osm = folium.Map(location=[33.48944, 126.48508], zoom_start= 13)
for i in s.index:
    folium.Circle([s.loc[i,'latitude'], s.loc[i,'longitude']], radius = 1,color='red', popup= s.loc[i,'bus_route_id']).add_to(map_osm)
map_osm

In [219]:
s = train.query('date == "2019-09-24" & bus_route_id == 23460000')
map_osm = folium.Map(location=[33.48944, 126.48508], zoom_start= 13)
for i in s.index:
    folium.Circle([s.loc[i,'latitude'], s.loc[i,'longitude']], radius = 1,color='red', popup= s.loc[i,'bus_route_id']).add_to(map_osm)
map_osm

In [220]:
s = train.query('date == "2019-09-25" & bus_route_id == 23460000')
map_osm = folium.Map(location=[33.48944, 126.48508], zoom_start= 13)
for i in s.index:
    folium.Circle([s.loc[i,'latitude'], s.loc[i,'longitude']], radius = 1,color='red', popup= s.loc[i,'bus_route_id']).add_to(map_osm)
map_osm

In [209]:
sorted(train.query('date == "2019-09-24" & bus_route_id == 23460000').station_name.unique())
# len(sorted(train.query('date == "2019-09-24" & bus_route_id == 23460000').station_name.unique()))

['거로입구',
 '고산동산(아라방면)',
 '광양',
 '국립제주박물관',
 '남국원(아라방면)',
 '대기고등학교',
 '도련1동',
 '도련2동',
 '도련반석스위트타운',
 '동문로터리(동문시장)',
 '매촌',
 '밤나무골',
 '봉개동주민센터',
 '삼성초등학교',
 '삼양2동',
 '삼양동주민센터',
 '삼양파출소사거리',
 '시민회관',
 '아라주공아파트',
 '아라초등학교',
 '아르미아파트',
 '오현중고등학교',
 '인다마을',
 '인화동',
 '인화동입구',
 '일도2동주민센터',
 '제대마을',
 '제대아파트',
 '제주대학교',
 '제주대학교병원',
 '제주대학교입구',
 '제주동중학교',
 '제주동초등학교',
 '제주시청(아라방면)',
 '제주여자상업고등학교',
 '제주여자중고등학교(아라방면)',
 '제주자동차검사소',
 '제주중앙여자고등학교(아라방면)',
 '제주지방법원(아라방면)',
 '중앙로(국민은행)',
 '천수동',
 '큰동네',
 '하나로아파트',
 '화북남문',
 '화북주공아파트',
 '화북초등학교']

In [210]:
sorted(train.query('date == "2019-09-25" & bus_route_id == 23460000').station_name.unique())
# len(sorted(train.query('date == "2019-09-25" & bus_route_id == 23460000').station_name.unique()))

['거로입구',
 '고산동산(아라방면)',
 '광양',
 '국립제주박물관',
 '남국원(아라방면)',
 '대기고등학교',
 '도련1동',
 '도련2동',
 '도련반석스위트타운',
 '동문로터리(동문시장)',
 '매촌',
 '봉개동주민센터',
 '삼성초등학교',
 '삼양2동',
 '삼양동주민센터',
 '삼양초등학교',
 '삼양파출소사거리',
 '시민회관',
 '아라주공아파트',
 '아라초등학교',
 '아르미아파트',
 '오현중고등학교',
 '인다마을',
 '인화동',
 '인화동입구',
 '일도2동주민센터',
 '제주대학교',
 '제주대학교교육대학',
 '제주대학교병원',
 '제주동중학교',
 '제주동초등학교',
 '제주시청(아라방면)',
 '제주여자상업고등학교',
 '제주여자중고등학교(아라방면)',
 '제주자동차검사소',
 '제주중앙여자고등학교(아라방면)',
 '제주지방법원(아라방면)',
 '죽성마을입구',
 '중앙로(국민은행)',
 '천수동',
 '큰동네',
 '하나로아파트',
 '화북남문',
 '화북주공아파트',
 '화북초등학교']

In [115]:
train['station_code'].value_counts()

358        1610
3270       1535
357        1515
3271       1482
3278       1423
320        1379
321        1334
3280       1297
3279       1272
150        1267
268        1238
3276       1202
1957       1175
3274       1169
291        1165
468        1143
121        1116
151        1114
3275       1107
3281       1104
292        1102
3277       1078
148        1070
360        1060
273        1060
164        1058
274        1057
3273       1052
3272       1051
125        1041
           ... 
1598       1   
734        1   
2182       1   
3037       1   
3461       1   
2914       1   
4038       1   
4239       1   
3411       1   
6115034    1   
3528       1   
4105       1   
4209       1   
3589       1   
4045       1   
1997       1   
2931       1   
4205       1   
3572       1   
1562       1   
1876       1   
4152       1   
2928       1   
2900       1   
3156       1   
4122       1   
4056       1   
1558       1   
3034       1   
3122       1   
Name: station_code, Leng

In [114]:
train.query(' station_name == "감귤유통센터"')

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,ride6,ride7,ride8,ride9,ride10,ride11,off6,off7,off8,off9,off10,off11,ride18,date_day,date_dayofweek,date_dayofname,holiday,dis_jejusi,dis_seoquipo
23564,23564,2019-09-02,30870000,0,4071,감귤유통센터,33.33214,126.80918,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,Monday,0.0,32.550114,24.726735
48835,48835,2019-09-04,27340000,0,4072,감귤유통센터,33.33230,126.80909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4,2,Wednesday,0.0,32.533048,24.724586
205471,205471,2019-09-16,27330000,0,4071,감귤유통센터,33.33214,126.80918,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16,0,Monday,0.0,32.550114,24.726735
383496,383496,2019-09-28,29290000,0,4071,감귤유통센터,33.33214,126.80918,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28,5,Saturday,1.0,32.550114,24.726735
406808,406808,2019-09-30,27330000,0,4071,감귤유통센터,33.33214,126.80918,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0,32.550114,24.726735
409948,409948,2019-09-30,28950000,0,4071,감귤유통센터,33.33214,126.80918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0,32.550114,24.726735


In [10]:
coords_jejusi = (33.500770, 126.522761) #제주시의 위도 경도
coords_seoquipo = (33.259429, 126.558217) #서귀포시의 위도 경도
train['dis_jejusi'] = [geopy.distance.vincenty((train['latitude'].iloc[i],train['longitude'].iloc[i]), coords_jejusi).km for i in range(len(train))]
train['dis_seoquipo'] = [geopy.distance.vincenty((train['latitude'].iloc[i],train['longitude'].iloc[i]), coords_seoquipo).km for i in range(len(train))]

# 메인

In [25]:
train.tail()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,ride6,ride7,ride8,ride9,ride10,ride11,off6,off7,off8,off9,off10,off11,ride18,date_day,date_dayofweek,date_dayofname,holiday,dis_jejusi,dis_seoquipo
415418,415418,2019-09-30,32820000,0,1129,한림환승정류장(한림리),33.41437,126.26336,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0,25.950139,32.385460
415419,415419,2019-09-30,32820000,0,1564,제주시외버스터미널,33.49946,126.51479,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0,0.754801,26.927069
415420,415420,2019-09-30,32820000,0,2322,해병부대,33.23100,126.26273,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0,38.482046,27.717482
415421,415421,2019-09-30,32820000,0,3291,애월환승정류장(애월리),33.46483,126.31870,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0,19.380000,31.873252
415422,415422,2019-09-30,32820000,0,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,30,0,Monday,0.0,27.987818,4.828453


In [23]:
train.loc[300000:len(train)]

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,ride6,ride7,ride8,ride9,ride10,ride11,off6,off7,off8,off9,off10,off11,ride18,date_day,date_dayofweek,date_dayofname,holiday,dis_jejusi,dis_seoquipo
300000,300000,2019-09-23,24050000,0,1480,가시나물,33.47109,126.57014,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,23,0,Monday,0.0,5.497779,23.501738
300001,300001,2019-09-23,24050000,0,1481,달샘마을,33.47451,126.57015,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23,0,Monday,0.0,5.280104,23.880686
300002,300002,2019-09-23,24050000,0,1482,달샘마을,33.47442,126.56996,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,23,0,Monday,0.0,5.270915,23.869898
300003,300003,2019-09-23,24050000,0,1499,제주영지학교,33.48929,126.54267,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23,0,Monday,0.0,2.245913,25.535162
300004,300004,2019-09-23,24050000,0,1536,화북초등학교후문,33.52023,126.56319,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,23,0,Monday,0.0,4.332260,28.929508
300005,300005,2019-09-23,24050000,0,1569,제주자동차검사정비사업조합,33.51029,126.56191,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23,0,Monday,0.0,3.787744,27.825448
300006,300006,2019-09-23,24050000,0,3278,제주여자중고등학교(아라방면),33.48486,126.54293,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,2.0,23,0,Monday,0.0,2.574287,25.043238
300007,300007,2019-09-23,24050000,0,3280,남국원(아라방면),33.47971,126.54381,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,23,0,Monday,0.0,3.046739,24.468358
300008,300008,2019-09-23,24050000,0,3323,영산홍아파트,33.49520,126.53993,0.0,4.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,23,0,Monday,0.0,1.710859,26.204951
300009,300009,2019-09-23,24050000,0,3325,탐라중학교,33.49085,126.54394,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,23,0,Monday,0.0,2.254772,25.701530


In [26]:
def get_station_across():
    
    d2 = train
#     d2 = train.loc[300000:len(train)] ## 데이터 확인용 
    blank = pd.DataFrame()
    
    for i in d2.index:
#         print(str(i)+"/"+ str(len(d2)))
        test = d2.loc[i]
        test_date = d2.loc[i, 'date']
        test_st_name = d2.loc[i, 'station_name']
        test_dis = d2.loc[i, 'dis_jejusi']       
        

        right = d2.query('date == @test_date & station_name == @test_st_name').reset_index()
        right['code'] = abs(right['station_code'] - d2.loc[i, 'station_code'])
        right['standard'] = abs(right['dis_jejusi'] - d2.loc[i, 'dis_jejusi'])
        right['route_id'] = abs(right['bus_route_id'] - d2.loc[i, 'bus_route_id'])

        bag = pd.DataFrame(np.zeros(len(right.columns))).T #26
        bag.columns = right.columns
        bag.station_name = right.station_name
        bag.station_code = right.station_code
        bag.date = right.date
        bag.in_out = right.in_out
        bag.bus_route_id = right.bus_route_id

        
        try:
            A = pd.DataFrame(test).T.merge(pd.DataFrame(right.sort_values(['route_id','code','standard']).reset_index(drop = True).loc[1]).T, on = 'station_name')
        except:
            A = pd.DataFrame(test).T.merge(bag, on = 'station_name')
        
        blank = pd.concat([blank, A])
    blank.drop(['code','standard','index','id_y'], axis = 1, inplace = True)
    blank.reset_index(inplace = True)
    return blank

In [28]:
new_train = get_station_across()
new_train.to_csv('newtrain5.csv')
new_train[['date_x','station_code_x','station_name','date_y','station_code_y']]

300000/115423
300001/115423
300002/115423
300003/115423
300004/115423
300005/115423
300006/115423
300007/115423
300008/115423
300009/115423
300010/115423
300011/115423
300012/115423
300013/115423
300014/115423
300015/115423
300016/115423
300017/115423
300018/115423
300019/115423
300020/115423
300021/115423
300022/115423
300023/115423
300024/115423
300025/115423
300026/115423
300027/115423
300028/115423
300029/115423
300030/115423
300031/115423
300032/115423
300033/115423
300034/115423
300035/115423
300036/115423
300037/115423
300038/115423
300039/115423
300040/115423
300041/115423
300042/115423
300043/115423
300044/115423
300045/115423
300046/115423
300047/115423
300048/115423
300049/115423
300050/115423
300051/115423
300052/115423
300053/115423
300054/115423
300055/115423
300056/115423
300057/115423
300058/115423
300059/115423
300060/115423
300061/115423
300062/115423
300063/115423
300064/115423
300065/115423
300066/115423
300067/115423
300068/115423
300069/115423
300070/115423
300071

300596/115423
300597/115423
300598/115423
300599/115423
300600/115423
300601/115423
300602/115423
300603/115423
300604/115423
300605/115423
300606/115423
300607/115423
300608/115423
300609/115423
300610/115423
300611/115423
300612/115423
300613/115423
300614/115423
300615/115423
300616/115423
300617/115423
300618/115423
300619/115423
300620/115423
300621/115423
300622/115423
300623/115423
300624/115423
300625/115423
300626/115423
300627/115423
300628/115423
300629/115423
300630/115423
300631/115423
300632/115423
300633/115423
300634/115423
300635/115423
300636/115423
300637/115423
300638/115423
300639/115423
300640/115423
300641/115423
300642/115423
300643/115423
300644/115423
300645/115423
300646/115423
300647/115423
300648/115423
300649/115423
300650/115423
300651/115423
300652/115423
300653/115423
300654/115423
300655/115423
300656/115423
300657/115423
300658/115423
300659/115423
300660/115423
300661/115423
300662/115423
300663/115423
300664/115423
300665/115423
300666/115423
300667

301182/115423
301183/115423
301184/115423
301185/115423
301186/115423
301187/115423
301188/115423
301189/115423
301190/115423
301191/115423
301192/115423
301193/115423
301194/115423
301195/115423
301196/115423
301197/115423
301198/115423
301199/115423
301200/115423
301201/115423
301202/115423
301203/115423
301204/115423
301205/115423
301206/115423
301207/115423
301208/115423
301209/115423
301210/115423
301211/115423
301212/115423
301213/115423
301214/115423
301215/115423
301216/115423
301217/115423
301218/115423
301219/115423
301220/115423
301221/115423
301222/115423
301223/115423
301224/115423
301225/115423
301226/115423
301227/115423
301228/115423
301229/115423
301230/115423
301231/115423
301232/115423
301233/115423
301234/115423
301235/115423
301236/115423
301237/115423
301238/115423
301239/115423
301240/115423
301241/115423
301242/115423
301243/115423
301244/115423
301245/115423
301246/115423
301247/115423
301248/115423
301249/115423
301250/115423
301251/115423
301252/115423
301253

301776/115423
301777/115423
301778/115423
301779/115423
301780/115423
301781/115423
301782/115423
301783/115423
301784/115423
301785/115423
301786/115423
301787/115423
301788/115423
301789/115423
301790/115423
301791/115423
301792/115423
301793/115423
301794/115423
301795/115423
301796/115423
301797/115423
301798/115423
301799/115423
301800/115423
301801/115423
301802/115423
301803/115423
301804/115423
301805/115423
301806/115423
301807/115423
301808/115423
301809/115423
301810/115423
301811/115423
301812/115423
301813/115423
301814/115423
301815/115423
301816/115423
301817/115423
301818/115423
301819/115423
301820/115423
301821/115423
301822/115423
301823/115423
301824/115423
301825/115423
301826/115423
301827/115423
301828/115423
301829/115423
301830/115423
301831/115423
301832/115423
301833/115423
301834/115423
301835/115423
301836/115423
301837/115423
301838/115423
301839/115423
301840/115423
301841/115423
301842/115423
301843/115423
301844/115423
301845/115423
301846/115423
301847

302363/115423
302364/115423
302365/115423
302366/115423
302367/115423
302368/115423
302369/115423
302370/115423
302371/115423
302372/115423
302373/115423
302374/115423
302375/115423
302376/115423
302377/115423
302378/115423
302379/115423
302380/115423
302381/115423
302382/115423
302383/115423
302384/115423
302385/115423
302386/115423
302387/115423
302388/115423
302389/115423
302390/115423
302391/115423
302392/115423
302393/115423
302394/115423
302395/115423
302396/115423
302397/115423
302398/115423
302399/115423
302400/115423
302401/115423
302402/115423
302403/115423
302404/115423
302405/115423
302406/115423
302407/115423
302408/115423
302409/115423
302410/115423
302411/115423
302412/115423
302413/115423
302414/115423
302415/115423
302416/115423
302417/115423
302418/115423
302419/115423
302420/115423
302421/115423
302422/115423
302423/115423
302424/115423
302425/115423
302426/115423
302427/115423
302428/115423
302429/115423
302430/115423
302431/115423
302432/115423
302433/115423
302434

...

413182/115423
413183/115423
413184/115423
413185/115423
413186/115423
413187/115423
413188/115423
413189/115423
413190/115423
413191/115423
413192/115423
413193/115423
413194/115423
413195/115423
413196/115423
413197/115423
413198/115423
413199/115423
413200/115423
413201/115423
413202/115423
413203/115423
413204/115423
413205/115423
413206/115423
413207/115423
413208/115423
413209/115423
413210/115423
413211/115423
413212/115423
413213/115423
413214/115423
413215/115423
413216/115423
413217/115423
413218/115423
413219/115423
413220/115423
413221/115423
413222/115423
413223/115423
413224/115423
413225/115423
413226/115423
413227/115423
413228/115423
413229/115423
413230/115423
413231/115423
413232/115423
413233/115423
413234/115423
413235/115423
413236/115423
413237/115423
413238/115423
413239/115423
413240/115423
413241/115423
413242/115423
413243/115423
413244/115423
413245/115423
413246/115423
413247/115423
413248/115423
413249/115423
413250/115423
413251/115423
413252/115423
413253

413768/115423
413769/115423
413770/115423
413771/115423
413772/115423
413773/115423
413774/115423
413775/115423
413776/115423
413777/115423
413778/115423
413779/115423
413780/115423
413781/115423
413782/115423
413783/115423
413784/115423
413785/115423
413786/115423
413787/115423
413788/115423
413789/115423
413790/115423
413791/115423
413792/115423
413793/115423
413794/115423
413795/115423
413796/115423
413797/115423
413798/115423
413799/115423
413800/115423
413801/115423
413802/115423
413803/115423
413804/115423
413805/115423
413806/115423
413807/115423
413808/115423
413809/115423
413810/115423
413811/115423
413812/115423
413813/115423
413814/115423
413815/115423
413816/115423
413817/115423
413818/115423
413819/115423
413820/115423
413821/115423
413822/115423
413823/115423
413824/115423
413825/115423
413826/115423
413827/115423
413828/115423
413829/115423
413830/115423
413831/115423
413832/115423
413833/115423
413834/115423
413835/115423
413836/115423
413837/115423
413838/115423
413839

414354/115423
414355/115423
414356/115423
414357/115423
414358/115423
414359/115423
414360/115423
414361/115423
414362/115423
414363/115423
414364/115423
414365/115423
414366/115423
414367/115423
414368/115423
414369/115423
414370/115423
414371/115423
414372/115423
414373/115423
414374/115423
414375/115423
414376/115423
414377/115423
414378/115423
414379/115423
414380/115423
414381/115423
414382/115423
414383/115423
414384/115423
414385/115423
414386/115423
414387/115423
414388/115423
414389/115423
414390/115423
414391/115423
414392/115423
414393/115423
414394/115423
414395/115423
414396/115423
414397/115423
414398/115423
414399/115423
414400/115423
414401/115423
414402/115423
414403/115423
414404/115423
414405/115423
414406/115423
414407/115423
414408/115423
414409/115423
414410/115423
414411/115423
414412/115423
414413/115423
414414/115423
414415/115423
414416/115423
414417/115423
414418/115423
414419/115423
414420/115423
414421/115423
414422/115423
414423/115423
414424/115423
414425

414940/115423
414941/115423
414942/115423
414943/115423
414944/115423
414945/115423
414946/115423
414947/115423
414948/115423
414949/115423
414950/115423
414951/115423
414952/115423
414953/115423
414954/115423
414955/115423
414956/115423
414957/115423
414958/115423
414959/115423
414960/115423
414961/115423
414962/115423
414963/115423
414964/115423
414965/115423
414966/115423
414967/115423
414968/115423
414969/115423
414970/115423
414971/115423
414972/115423
414973/115423
414974/115423
414975/115423
414976/115423
414977/115423
414978/115423
414979/115423
414980/115423
414981/115423
414982/115423
414983/115423
414984/115423
414985/115423
414986/115423
414987/115423
414988/115423
414989/115423
414990/115423
414991/115423
414992/115423
414993/115423
414994/115423
414995/115423
414996/115423
414997/115423
414998/115423
414999/115423
415000/115423
415001/115423
415002/115423
415003/115423
415004/115423
415005/115423
415006/115423
415007/115423
415008/115423
415009/115423
415010/115423
415011

,date_x,station_code_x,station_name,date_y,station_code_y
0,2019-09-23 00:00:00,1480,가시나물,2019-09-23 00:00:00,1480
1,2019-09-23 00:00:00,1481,달샘마을,2019-09-23 00:00:00,1482
2,2019-09-23 00:00:00,1482,달샘마을,2019-09-23 00:00:00,1481
3,2019-09-23 00:00:00,1499,제주영지학교,2019-09-23 00:00:00,1499
4,2019-09-23 00:00:00,1536,화북초등학교후문,2019-09-23 00:00:00,1536
5,2019-09-23 00:00:00,1569,제주자동차검사정비사업조합,2019-09-23 00:00:00,1569
6,2019-09-23 00:00:00,3278,제주여자중고등학교(아라방면),2019-09-23 00:00:00,3278
7,2019-09-23 00:00:00,3280,남국원(아라방면),2019-09-23 00:00:00,3280
8,2019-09-23 00:00:00,3323,영산홍아파트,2019-09-23 00:00:00,3324
9,2019-09-23 00:00:00,3325,탐라중학교,2019-09-23 00:00:00,3326


In [101]:
#확인을 위해 station_code_y와 latitude_y, longitude_y 칼럼을 남겼습니다. OLS 직전에 삭제하시면 될 것 같습니다.
new_train.tail()

,index,id_x,date_x,bus_route_id_x,in_out_x,station_code_x,station_name,latitude_x,longitude_x,ride6_x,ride7_x,ride8_x,ride9_x,ride10_x,ride11_x,off6_x,off7_x,off8_x,off9_x,off10_x,off11_x,ride18_x,date_day_x,date_dayofweek_x,date_dayofname_x,holiday_x,dis_jejusi_x,dis_seoquipo_x,date_y,bus_route_id_y,in_out_y,station_code_y,latitude_y,longitude_y,ride6_y,ride7_y,ride8_y,ride9_y,ride10_y,ride11_y,off6_y,off7_y,off8_y,off9_y,off10_y,off11_y
496,0,496,2019-09-01 00:00:00,21610000,0,478,대림2차아파트,33.479,126.488,0,0,0,0,0,0,1,0,0,0,0,1,0,1,6,Sunday,1,4.03312,25.2143,2019-09-01 00:00:00,21610000,0,478,0,0,0,0,0,0,0,0,0,0,0,0,0,0
497,0,497,2019-09-01 00:00:00,21610000,0,480,연동대림1차아파트,33.4803,126.49,0,0,0,0,0,0,0,0,1,0,0,0,0,1,6,Sunday,1,3.81787,25.31,2019-09-01 00:00:00,21610000,0,481,33.4807,126.49,0,0,0,1,0,0,0,0,0,0,0,0
498,0,498,2019-09-01 00:00:00,21610000,0,481,연동대림1차아파트,33.4807,126.49,0,0,0,1,0,0,0,0,0,0,0,0,1,1,6,Sunday,1,3.7768,25.3508,2019-09-01 00:00:00,21610000,0,480,33.4803,126.49,0,0,0,0,0,0,0,0,1,0,0,0
499,0,499,2019-09-01 00:00:00,21610000,0,520,오름가름,33.4716,126.508,0,0,0,0,0,0,0,0,0,0,1,0,0,1,6,Sunday,1,3.5138,23.9995,2019-09-01 00:00:00,21610000,0,520,0,0,0,0,0,0,0,0,0,0,0,0,0,0
500,0,500,2019-09-01 00:00:00,21610000,0,533,오등동,33.4751,126.529,0,1,1,0,1,0,0,0,0,0,0,0,1,1,6,Sunday,1,2.90909,24.0745,2019-09-01 00:00:00,21610000,0,533,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [103]:
train.query('date == "2019-09-01" & station_name == "중앙로터리(동)"')

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,ride6,ride7,ride8,ride9,ride10,ride11,off6,off7,off8,off9,off10,off11,ride18,date_day,date_dayofweek,date_dayofname,holiday,dis_jejusi,dis_seoquipo
125,125,2019-09-01,21010000,0,1784,중앙로터리(동),33.25297,126.56135,1.0,0.0,3.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,6,Sunday,1.0,27.717377,0.773565
451,451,2019-09-01,21520000,0,1784,중앙로터리(동),33.25297,126.56135,0.0,8.0,4.0,10.0,1.0,3.0,0.0,1.0,1.0,4.0,1.0,6.0,6.0,1,6,Sunday,1.0,27.717377,0.773565
467,467,2019-09-01,21540000,0,1784,중앙로터리(동),33.25297,126.56135,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,27.717377,0.773565
474,474,2019-09-01,21550000,0,1784,중앙로터리(동),33.25297,126.56135,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,27.717377,0.773565
556,556,2019-09-01,21910000,0,1784,중앙로터리(동),33.25297,126.56135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,1,6,Sunday,1.0,27.717377,0.773565
569,569,2019-09-01,21920000,0,1784,중앙로터리(동),33.25297,126.56135,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6,Sunday,1.0,27.717377,0.773565
4905,4905,2019-09-01,25000000,0,1784,중앙로터리(동),33.25297,126.56135,1.0,5.0,10.0,16.0,8.0,9.0,0.0,1.0,2.0,7.0,3.0,3.0,20.0,1,6,Sunday,1.0,27.717377,0.773565
5132,5132,2019-09-01,25060000,0,1784,중앙로터리(동),33.25297,126.56135,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,1,6,Sunday,1.0,27.717377,0.773565
5165,5165,2019-09-01,25070000,0,1784,중앙로터리(동),33.25297,126.56135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,6,Sunday,1.0,27.717377,0.773565
5206,5206,2019-09-01,25080000,0,1784,중앙로터리(동),33.25297,126.56135,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,4.0,0.0,3.0,1,6,Sunday,1.0,27.717377,0.773565


In [102]:
new_train.tail(50)

,index,id_x,date_x,bus_route_id_x,in_out_x,station_code_x,station_name,latitude_x,longitude_x,ride6_x,ride7_x,ride8_x,ride9_x,ride10_x,ride11_x,off6_x,off7_x,off8_x,off9_x,off10_x,off11_x,ride18_x,date_day_x,date_dayofweek_x,date_dayofname_x,holiday_x,dis_jejusi_x,dis_seoquipo_x,date_y,bus_route_id_y,in_out_y,station_code_y,latitude_y,longitude_y,ride6_y,ride7_y,ride8_y,ride9_y,ride10_y,ride11_y,off6_y,off7_y,off8_y,off9_y,off10_y,off11_y
451,0,451,2019-09-01 00:00:00,21520000,0,1784,중앙로터리(동),33.253,126.561,0,8,4,10,1,3,0,1,1,4,1,6,6,1,6,Sunday,1,27.7174,0.773565,2019-09-01 00:00:00,21540000,0,1784,33.253,126.561,0,1,0,0,0,0,0,0,0,0,0,0
452,0,452,2019-09-01 00:00:00,21520000,0,1869,서귀포산업과학고등학교,33.296,126.596,1,1,1,0,0,0,0,0,1,0,0,0,1,1,6,Sunday,1,23.7031,5.34168,2019-09-01 00:00:00,21520000,0,1869,0,0,0,0,0,0,0,0,0,0,0,0,0,0
453,0,453,2019-09-01 00:00:00,21520000,0,1919,비석거리,33.2558,126.577,0,14,16,1,4,4,0,0,3,0,2,4,7,1,6,Sunday,1,27.6446,1.83727,2019-09-01 00:00:00,21540000,0,1919,33.2558,126.577,0,2,0,0,0,0,0,0,0,0,0,0
454,0,454,2019-09-01 00:00:00,21520000,0,2052,중문환승정류장(중문우체국),33.2518,126.424,1,0,2,0,1,0,1,2,3,1,5,2,2,1,6,Sunday,1,29.1086,12.5539,2019-09-01 00:00:00,21500000,0,2053,33.2519,126.423,1,3,0,0,0,0,0,0,0,0,0,0
455,0,455,2019-09-01 00:00:00,21520000,0,2106,서귀포여자중학교,33.2543,126.547,0,0,0,0,0,2,0,0,0,0,2,2,2,1,6,Sunday,1,27.4304,1.18888,2019-09-01 00:00:00,21480000,0,2107,33.2549,126.546,0,0,5,0,0,0,0,0,0,0,0,0
456,0,456,2019-09-01 00:00:00,21520000,0,2494,동광환승정류장5(서귀방면),33.3096,126.341,0,0,0,0,2,0,0,1,1,5,0,3,3,1,6,Sunday,1,27.1308,21.0001,2019-09-01 00:00:00,4270000,0,2494,33.3096,126.341,0,0,0,0,0,0,0,0,0,0,3,2
457,0,457,2019-09-01 00:00:00,21520000,0,2972,제주국제공항(종점),33.5057,126.493,0,0,0,0,0,0,0,3,21,12,6,11,0,1,6,Sunday,1,2.80181,27.9775,2019-09-01 00:00:00,21500000,0,2972,33.5057,126.493,0,0,0,0,0,0,0,14,3,0,0,0
458,0,458,2019-09-01 00:00:00,21520000,0,3233,토평사거리,33.2688,126.586,0,1,1,2,2,0,0,0,0,0,0,0,0,1,6,Sunday,1,26.3807,2.75534,2019-09-01 00:00:00,21480000,0,3232,33.2688,126.585,0,0,0,0,0,0,0,0,0,0,1,0
459,0,459,2019-09-01 00:00:00,21520000,0,3271,제주시청(광양방면),33.4989,126.53,0,0,0,0,1,0,0,0,3,1,3,0,3,1,6,Sunday,1,0.734436,26.6885,2019-09-01 00:00:00,21540000,0,3271,33.4989,126.53,0,0,0,0,0,0,0,0,1,0,0,0
460,0,460,2019-09-01 00:00:00,21520000,0,3275,제주지방법원(광양방면),33.4936,126.535,0,0,0,0,0,0,0,0,0,0,1,0,0,1,6,Sunday,1,1.36762,26.0671,2019-09-01 00:00:00,21540000,0,3275,33.4936,126.535,0,0,0,0,0,0,0,0,1,0,0,0
